# Setting Up Environment

In [1]:
import os
import json
import math
import logging
import networkx as nx
import numpy as np
import pandas as pd

from collections import Counter, defaultdict
from itertools import product
from typing import Mapping, List
from tqdm import tqdm
from scipy import stats

from utils import DATA_DIR, KG_DATA_PATH, create_graph_from_df

# Load Graph and Gold Standard

In [2]:
openbiolink_df = pd.read_csv(
    os.path.join(KG_DATA_PATH, 'normalized', 'openbiolink_kg_normalized.tsv'),
    sep='\t'
)
openbiolink_df.rename(columns={'relation': 'polarity'}, inplace=True)

custom_df = pd.read_csv(
    os.path.join(KG_DATA_PATH, 'normalized', 'custom_kg_normalized.tsv'),
    sep='\t'
)
custom_df.rename(columns={'relation': 'polarity'}, inplace=True)

# Load clinical data

In [3]:
with open(os.path.join(DATA_DIR, 'gold-standard', 'filtered-clinical-pairs.json')) as file:
    clinical_pair_dict = json.load(file).keys()

In [4]:
gold_standard_dict = {}
for i in clinical_pair_dict:
    split = i.split("_")
    if split[1] in gold_standard_dict:
        gold_standard_dict[split[1]].append(split[0])
    else:
        gold_standard_dict[split[1]] = [split[0]]

# Create KG 

In [5]:
graph_openbio = create_graph_from_df(openbiolink_df)
graph_custom = create_graph_from_df(custom_df)

Report on the number of relations: {-1: 16133, 1: 32745}
Report on the number of relations: {1: 43810, -1: 8372}


# Create drug disease dicts specific to KG

In [6]:
drug_openbio = set(openbiolink_df.loc[openbiolink_df['source'].str.contains('pubchem.compound')]['source'].unique())
disease_openbio = set(openbiolink_df.loc[openbiolink_df['target'].str.contains('mondo')]['target'].unique())

In [7]:
drug_custom = set(custom_df.loc[custom_df['source'].str.contains('pubchem.compound')]['source'].unique())
disease_custom = set(custom_df.loc[custom_df['target'].str.contains('mondo')]['target'].unique())

# Helper Functions

In [8]:
score_actual = {}
kg_dfs = {}

In [9]:
def khop(
    nodeA: str, 
    nodeB: str, 
    graph: nx.Graph, 
    limit: int,
    total: bool
) -> tuple:
    
    """Find nodes within the distance limit """
    traversalA = nx.bfs_edges(graph, source=nodeA, depth_limit=limit)
    traversalB = nx.bfs_edges(graph, source=nodeB, depth_limit=limit)
    
    khop_A = set([v for u, v in traversalA])
    khop_B = set([v for u, v in traversalB])
    
    if total:
        return list(khop_A | khop_B), khop_A, khop_B
    else:
        return list(khop_A & khop_B), khop_A, khop_B

In [10]:
def get_dict_df(
    diseases, 
    drugs, 
    undirected_kg_graph, 
    similarity_type
):
    print(similarity_type)
    
    df = pd.DataFrame(columns=[
        'Source',
        'Target',
        sim_scores[similarity_type]
    ])
    
    disease_drug_dict = {}
    
    for disease in diseases:
        # Skip drugs not part of largest component of KG
        if disease not in undirected_kg_graph.nodes():
            continue
        
        cn = []
        
        # for each disease, find the similarity score with for each drug and append to list
        for drug in drugs:
            
            # Skip drugs not part of largest component of KG
            if drug not in undirected_kg_graph.nodes():
                continue
            
            shared_nodes, nodeA_neighbor, nodeB_neighbor = khop(
                nodeA=drug,
                nodeB=disease,
                graph=undirected_kg_graph, 
                limit=1,
                total=False,
            )
            
            total_nodes, _, _ = khop(
                nodeA=drug,
                nodeB=disease,
                graph=undirected_kg_graph, 
                limit=1,
                total=True,
            )
                
            if similarity_type == 'cn':
                similarity = len(shared_nodes)
            
            elif similarity_type == 'cos':
                similarity = len(shared_nodes) / (math.sqrt(len(nodeA_neighbor) * len(nodeB_neighbor)))
            
            elif similarity_type == 'ji':
                similarity = len(shared_nodes) / len(total_nodes)
                
            elif similarity_type == 'si':
                similarity = (2 * len(shared_nodes)) / (len(nodeA_neighbor) + len(nodeB_neighbor))
                
            elif similarity_type == 'hpi':
                similarity = len(shared_nodes) / min(len(nodeA_neighbor), len(nodeB_neighbor)) 
                
            elif similarity_type == 'hdi':
                similarity = len(shared_nodes) / max(len(nodeA_neighbor), len(nodeB_neighbor)) 
                
            elif similarity_type == 'lhn':
                similarity = len(shared_nodes) / (len(nodeA_neighbor) * len(nodeB_neighbor)) 
                
            elif similarity_type == 'pa':
                similarity = len(nodeA_neighbor) * len(nodeB_neighbor)
                
            elif similarity_type == 'aa':
                similarity = 0
                
                for n in shared_nodes:
                    neighbors_list = set(i for i in undirected_kg_graph.neighbors(n))
                    similarity += 1 / math.log10(len(neighbors_list))
                    
            elif similarity_type == 'ra':
                similarity = 0
                
                for n in shared_nodes:
                    neighbors_list = set(i for i in undirected_kg_graph.neighbors(n))
                    similarity += 1 / len(neighbors_list)
            
            cn.append(similarity)
            
        
        index = np.where(cn == np.amax(cn))
        # if list is full of 0's (i.e sum == 0), then there are no shared neighbors 
        
        if np.sum(cn) == 0 or len(np.array(cn)[index]) > 1:
            disease_drug_dict[disease] = 'none'          
        else:
            # add list of drugs with max # of shared nodes to specified key
            disease_drug_dict[disease] = np.array(list(drugs))[index]
                
            for val in index:
                j = val[0]
                df = df.append(
                    {
                        'Source': disease, 
                        'Target': list(drugs)[j], 
                        sim_scores[similarity_type]: cn[j]
                    }, 
                    ignore_index=True
                )

    return disease_drug_dict, df

In [11]:
def get_precision(
    clinical_trial_dict: dict, 
    predicted_dict: dict
)-> tuple: 
    pos, total = 0,0
    for disease in predicted_dict:
        # TODO: Check again...
        if disease not in clinical_trial_dict:
            total += 1
            continue
            
        for drug in predicted_dict[disease]:
            if drug in clinical_trial_dict[disease]:
                pos+=1
                
        total+=1
    return round(((pos/total) * 100), 3), pos, total


# Value by change for both KGs

In [12]:
openbio_prob = 0

for disease in disease_openbio:
    for drug in drug_openbio:
        trial = f'{drug}_{disease}'
        if trial in clinical_pair_dict:
            openbio_prob += 1

total = len(drug_openbio) * len(disease_openbio)
prob = openbio_prob / total
round(prob, 3)

0.016

In [13]:
custom_prob = 0

for disease in disease_custom:
    for drug in drug_custom:
        trial = f'{drug}_{disease}'
        if trial in clinical_pair_dict:
            custom_prob += 1

total = len(drug_custom) * len(disease_custom)
prob = custom_prob / total
round(prob, 3)

0.046

# Get P values for each graph

In [14]:
def get_pvalue(positive, total, prob):
    return stats.binom_test(
        x=positive,
        n=total,
        p=prob,
        alternative='greater'
    )

# Different benchmark methods

In [15]:
sim_scores = {
    'cn': 'Common Neighbors',
    'cos': 'Cosine Similiarity',
    'ji': 'Jaccard index',
    'si': 'Sorensen index',
    'hpi': 'Hub Promoted Index',
    'hdi': 'Hub Depressed Index', 
    'lhn': 'Leicht–Holme–Newman Index',
    'pa':'Preferential Attachment',
    'aa': 'Adamic-Adar', 
    'ra': 'Resource Allocation Index', 
}

In [16]:
for algo in sim_scores:
    # OpenBioLink KG
    data_dict_openbio, df_openbio = get_dict_df(
        disease_openbio, 
        drug_openbio, 
        graph_openbio.to_undirected(),
        similarity_type=algo
    )
    
    algo_name = sim_scores[algo]
    
    openbio_accuracy, openbio_pos, openbio_total = get_precision(
        clinical_trial_dict=gold_standard_dict, 
        predicted_dict=data_dict_openbio
    )
    
    openbio_p_val = get_pvalue(
        positive=openbio_pos,
        total=openbio_total,
        prob=0.016
    )
    
    # Custom KG
    data_dict_cutom, df_custom = get_dict_df(
        disease_custom, 
        drug_custom, 
        graph_custom.to_undirected(),
        similarity_type=algo
    )
    
    custom_accuracy, custom_pos, custom_total = get_precision(
        gold_standard_dict, 
        data_dict_cutom
    )
    
    custom_p_val = get_pvalue(
        positive=custom_pos,
        total=custom_total,
        prob=0.046
    )
   
    score_actual[algo_name] = {
        'openbio_precision': openbio_accuracy,
        'openbio_val_by_chance': '0.016',
        'openbio_p_val': openbio_p_val,
        'custom_precision': custom_accuracy,
        'custom_val_by_chance': '0.046',
        'custom_p_val': custom_p_val,
    }    

cn
cn
cos
cos
ji
ji
si
si
hpi
hpi
hdi
hdi
lhn
lhn
pa
pa
aa
aa
ra
ra


In [17]:
pd.DataFrame(score_actual).transpose()

,openbio_precision,openbio_val_by_chance,openbio_p_val,custom_precision,custom_val_by_chance,custom_p_val
Common Neighbors,0,0.016,1,0,0.046,1
Cosine Similiarity,2.778,0.016,0.3204,1.695,0.046,0.937862
Jaccard index,2.778,0.016,0.3204,1.695,0.046,0.937862
Sorensen index,2.778,0.016,0.3204,1.695,0.046,0.937862
Hub Promoted Index,0,0.016,1,1.695,0.046,0.937862
Hub Depressed Index,2.778,0.016,0.3204,0,0.046,1
Leicht–Holme–Newman Index,2.778,0.016,0.3204,1.695,0.046,0.937862
Preferential Attachment,2.778,0.016,0.3204,0,0.046,1
Adamic-Adar,0,0.016,1,0,0.046,1
Resource Allocation Index,0,0.016,1,0,0.046,1
